In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
import optuna
import warnings

warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Load both datasets
df_2025 = pd.read_csv('../data/master_features_all.csv')
df_2026 = pd.read_csv('../data/features_2026.csv')

print(f"2025 dataset: {df_2025.shape}")
print(f"2026 dataset: {df_2026.shape}")
print(f"\n2025 columns:\n{df_2025.columns.tolist()}")
print(f"\n2026 columns:\n{df_2026.columns.tolist()}")

2025 dataset: (1398, 27)
2026 dataset: (22, 19)

2025 columns:
['year', 'round', 'race_name', 'circuit', 'date', 'driver', 'driver_name', 'team', 'grid_position', 'finish_position', 'outperformance', 'dnf', 'rolling_avg_finish_5', 'rolling_avg_points_5', 'constructor_rolling_points_5', 'dnf_rate', 'driver_experience', 'season_stage', 'circuit_type', 'overtaking_difficulty', 'safety_car_probability', 'is_sprint_weekend', 'avg_team_pit_seconds', 'quali_position', 'rolling_avg_quali_5', 'teammate_quali_gap', 'sample_weight']

2026 columns:
['driver', 'test1_best_s', 'test2_best_s', 'test1_gap_s', 'test2_gap_s', 'testing_improvement_s', 'combined_pace_gap_s', 'team_2026', 'total_testing_laps', 'avg_race_pace_s', 'race_pace_gap_s', 'team_total_laps', 'team_reliability_score', 'driver_team_change', 'new_team_flag', 'rookie_flag', 'barcelona_best_s', 'barcelona_gap_s', 'missed_barcelona']


In [2]:
# FEATURE PREP + TRAIN/TEST SPLIT

TARGET = 'finish_position'

META_COLS = [
    'year', 'race_name', 'circuit', 
    'date', 'driver', 'driver_name', 'team'
]

LEAKAGE_COLS = [
    'status',          
    'points',             
    'laps_completed',      
    'fastest_lap_rank',    
    'dnf',                 
    'teammate_finish_gap',
    'championship_gap',    
    'wet_race',
    'sample_weight',       # not a feature, used for training only
]

# ONE HOT ENCODE circuit_type and season_stage
df_encoded = pd.get_dummies(
    df_2025, 
    columns=['circuit_type', 'season_stage'], 
    prefix=['circuit', 'stage']
)

# SPLIT FIRST — use 2025 data only for train/test split evaluation
# Train = 2023 + 2024 + 2025 rounds 1-18
# Test  = 2025 rounds 19-24
train_mask = ~((df_encoded['year'] == 2025) & (df_encoded['round'] >= 19))
test_mask  =   (df_encoded['year'] == 2025) & (df_encoded['round'] >= 19)

# NOW drop metadata + leakage + round
drop_cols = [c for c in META_COLS + LEAKAGE_COLS + ['round'] 
             if c in df_encoded.columns]
df_encoded = df_encoded.drop(columns=drop_cols)

# Feature columns = everything except target
feature_cols = [c for c in df_encoded.columns if c != TARGET]

# Redundant/useless features based on feature importance
WEAK_FEATURES = ['home_race', 'circuit_permanent', 'circuit_street', 'stage_late']
feature_cols = [c for c in feature_cols if c not in WEAK_FEATURES]

print(f"Target: {TARGET}")
print(f"Number of features: {len(feature_cols)}")
print(f"\nFeatures:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

df_train = df_encoded[train_mask].copy()
df_test  = df_encoded[test_mask].copy()

X_train = df_train[feature_cols]
y_train = df_train[TARGET]
w_train = df_2025[train_mask]['sample_weight']

X_test = df_test[feature_cols]
y_test = df_test[TARGET]

print(f"\nTRAIN / TEST SPLIT:")
print(f"Training rows : {len(X_train)} (2023+2024+2025 Rounds 1-18)")
print(f"Test rows     : {len(X_test)} (2025 Rounds 19-24)")


Target: finish_position
Number of features: 16

Features:
  1. grid_position
  2. outperformance
  3. rolling_avg_finish_5
  4. rolling_avg_points_5
  5. constructor_rolling_points_5
  6. dnf_rate
  7. driver_experience
  8. overtaking_difficulty
  9. safety_car_probability
  10. is_sprint_weekend
  11. avg_team_pit_seconds
  12. quali_position
  13. rolling_avg_quali_5
  14. teammate_quali_gap
  15. stage_early
  16. stage_mid

TRAIN / TEST SPLIT:
Training rows : 1278 (2023+2024+2025 Rounds 1-18)
Test rows     : 120 (2025 Rounds 19-24)


In [3]:
# TRAIN BASELINE LIGHTGBM MODEL

model = lgb.LGBMRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    num_leaves=31,
    min_child_samples=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    verbose=-1
)

model.fit(X_train, y_train, sample_weight=w_train)
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.3f} positions off on average")
print(f"  e.g. if driver actually finishes P5,")
print(f"  we predict somewhere around P{5-mae:.0f} to P{5+mae:.0f}")

results_df = pd.DataFrame({
    'actual'   : y_test.values,
    'predicted': y_pred.round(1),
    'error'    : (y_pred - y_test.values).round(1)
}).reset_index(drop=True)

print(f"\nSAMPLE PREDICTIONS (first 20 rows):")
print(results_df.head(20).to_string())


MAE: 0.459 positions off on average
  e.g. if driver actually finishes P5,
  we predict somewhere around P5 to P5

SAMPLE PREDICTIONS (first 20 rows):
    actual  predicted  error
0       14       13.8   -0.2
1       12       12.1    0.1
2       11       11.1    0.1
3       16       15.7   -0.3
4       11       11.3    0.3
5       16       15.6   -0.4
6       13       13.7    0.7
7        6        5.2   -0.8
8        2        2.4    0.4
9        3        4.1    1.1
10       5        5.3    0.3
11      15       14.2   -0.8
12      20       19.4   -0.6
13      17       15.6   -1.4
14      13       12.9   -0.1
15       5        5.8    0.8
16       3        3.7    0.7
17      13       12.6   -0.4
18       3        3.3    0.3
19       2        2.0    0.0


In [4]:
# FEATURE IMPORTANCE

importance_df = pd.DataFrame({
    'feature'   : feature_cols,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False).reset_index(drop=True)

print("FEATURE IMPORTANCE:")
print(importance_df.to_string())

FEATURE IMPORTANCE:
                         feature  importance
0                 outperformance        1762
1                  grid_position        1073
2           avg_team_pit_seconds         825
3                 quali_position         735
4            rolling_avg_quali_5         682
5           rolling_avg_finish_5         553
6           rolling_avg_points_5         438
7              driver_experience         421
8             teammate_quali_gap         402
9   constructor_rolling_points_5         383
10                      dnf_rate         378
11        safety_car_probability         239
12         overtaking_difficulty          79
13             is_sprint_weekend          63
14                     stage_mid          53
15                   stage_early          49


In [5]:
# OPTUNA HYPERPARAMETER TUNING (single split)

def objective(trial):
    params = {
        'n_estimators'     : trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate'    : trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth'        : trial.suggest_int('max_depth', 3, 10),
        'num_leaves'       : trial.suggest_int('num_leaves', 15, 127),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'subsample'        : trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'random_state'     : 42,
        'verbose'          : -1
    }
    m = lgb.LGBMRegressor(**params)
    m.fit(X_train, y_train, sample_weight=w_train)
    y_pred = m.predict(X_test)
    return mean_absolute_error(y_test, y_pred)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300, show_progress_bar=True)
print(f"Best MAE : {study.best_value:.3f}")
print(f"Best params:\n{study.best_params}")


  0%|          | 0/300 [00:00<?, ?it/s]

Best MAE : 0.380
Best params:
{'n_estimators': 959, 'learning_rate': 0.04575664966316788, 'max_depth': 3, 'num_leaves': 31, 'min_child_samples': 6, 'subsample': 0.9588949742036352, 'colsample_bytree': 0.8180684991304144}


Baseline (default params)  →  0.635 MAE
After dropping weak features →  0.512 MAE
After Optuna tuning          →  0.438 MAE

Total improvement: 0.197 positions ✅

learning_rate = 0.125   → higher than our 0.05 guess
max_depth = 10          → deeper trees than we used
num_leaves = 22         → fewer leaves than default 31
n_estimators = 457      → more trees than our 300

In [6]:
# RETRAIN WITH BEST PARAMS

best_model = lgb.LGBMRegressor(**study.best_params, random_state=42, verbose=-1)
best_model.fit(X_train, y_train, sample_weight=w_train)

y_pred_best = best_model.predict(X_test)
mae_best = mean_absolute_error(y_test, y_pred_best)

print(f"Final MAE: {mae_best:.3f} positions off on average")

results_df = pd.DataFrame({
    'actual'   : y_test.values,
    'predicted': y_pred_best.round(1),
    'error'    : (y_pred_best - y_test.values).round(1)
}).reset_index(drop=True)

print(f"\nSample predictions (first 20 rows):")
print(results_df.head(20).to_string())


Final MAE: 0.380 positions off on average

Sample predictions (first 20 rows):
    actual  predicted  error
0       14       13.7   -0.3
1       12       12.1    0.1
2       11       11.2    0.2
3       16       15.8   -0.2
4       11       11.3    0.3
5       16       16.0    0.0
6       13       13.4    0.4
7        6        5.3   -0.7
8        2        2.5    0.5
9        3        4.0    1.0
10       5        4.9   -0.1
11      15       15.6    0.6
12      20       19.3   -0.7
13      17       15.9   -1.1
14      13       12.8   -0.2
15       5        5.5    0.5
16       3        4.1    1.1
17      13       12.3   -0.7
18       3        3.1    0.1
19       2        1.7   -0.3


# OPTUNA SINGLE SPLIT (baseline tuning, reference only)
# Note: this overfit to Rounds 19-24, replaced by Cell 8 CV tuning

In [20]:
# CROSS VALIDATION — ROUND-BASED PER YEAR

folds = [
    (df_2025[df_2025['year'] == 2023]['round'].isin(range(1, 19)),
     df_2025[df_2025['year'] == 2023]['round'].isin(range(19, 23))),
    (df_2025[df_2025['year'] == 2024]['round'].isin(range(1, 19)),
     df_2025[df_2025['year'] == 2024]['round'].isin(range(19, 25))),
    (df_2025[df_2025['year'] == 2025]['round'].isin(range(1, 19)),
     df_2025[df_2025['year'] == 2025]['round'].isin(range(19, 25))),
]

fold_maes = []

for i, (train_idx, test_idx) in enumerate(folds, 1):
    year = [2023, 2024, 2025][i-1]

    df_cv = pd.get_dummies(df_2025[df_2025['year'] == year],
                           columns=['circuit_type', 'season_stage'],
                           prefix=['circuit', 'stage'])
    drop_cols_cv = [c for c in META_COLS + LEAKAGE_COLS + ['round']
                    if c in df_cv.columns]
    df_cv = df_cv.drop(columns=drop_cols_cv)

    X_tr = df_cv[train_idx][feature_cols]
    y_tr = df_cv[train_idx][TARGET]
    w_tr = df_2025[(df_2025['year'] == year)][train_idx]['sample_weight']
    X_te = df_cv[test_idx][feature_cols]
    y_te = df_cv[test_idx][TARGET]

    cv_model = lgb.LGBMRegressor(**study.best_params, random_state=42, verbose=-1)
    cv_model.fit(X_tr, y_tr, sample_weight=w_tr)

    train_mae = mean_absolute_error(y_tr, cv_model.predict(X_tr))
    test_mae  = mean_absolute_error(y_te, cv_model.predict(X_te))
    gap       = test_mae - train_mae

    fold_maes.append(test_mae)
    print(f"Fold {i} ({year}) | Train MAE: {train_mae:.3f} | Test MAE: {test_mae:.3f} | Gap: {gap:.3f}")

print(f"\nAverage test MAE : {np.mean(fold_maes):.3f}")
print(f"Std deviation    : {np.std(fold_maes):.3f}")
print(f"\nIf gap is small (<0.2) → healthy fit")
print(f"If gap is large (>0.5) → overfitting")

Fold 1 (2023) | Train MAE: 0.099 | Test MAE: 0.669 | Gap: 0.570
Fold 2 (2024) | Train MAE: 0.069 | Test MAE: 0.766 | Gap: 0.697
Fold 3 (2025) | Train MAE: 0.105 | Test MAE: 0.467 | Gap: 0.362

Average test MAE : 0.634
Std deviation    : 0.125

If gap is small (<0.2) → healthy fit
If gap is large (>0.5) → overfitting


In [21]:
# OPTUNA WITH CROSS VALIDATION OBJECTIVE (year-based folds)

def cv_objective(trial):
    params = {
        'n_estimators'     : trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate'    : trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth'        : trial.suggest_int('max_depth', 3, 6),
        'num_leaves'       : trial.suggest_int('num_leaves', 15, 63),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50),
        'subsample'        : trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha'        : trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda'       : trial.suggest_float('reg_lambda', 0.0, 1.0),
        'random_state'     : 42,
        'verbose'          : -1
    }

    fold_maes = []
    for year, test_end in [(2023, 23), (2024, 25), (2025, 25)]:
        year_mask  = df_2025['year'] == year
        train_idx  = year_mask & df_2025['round'].isin(range(1, 19))
        test_idx   = year_mask & df_2025['round'].isin(range(19, test_end))

        df_cv = pd.get_dummies(df_2025[year_mask],
                               columns=['circuit_type', 'season_stage'],
                               prefix=['circuit', 'stage'])
        drop_cols_cv = [c for c in META_COLS + LEAKAGE_COLS + ['round']
                        if c in df_cv.columns]
        df_cv = df_cv.drop(columns=drop_cols_cv)

        # Realign index for boolean mask
        train_idx_local = df_2025[year_mask]['round'].isin(range(1, 19)).values
        test_idx_local  = df_2025[year_mask]['round'].isin(range(19, test_end)).values

        X_tr = df_cv[train_idx_local][feature_cols]
        y_tr = df_cv[train_idx_local][TARGET]
        w_tr = df_2025[year_mask][train_idx_local]['sample_weight']
        X_te = df_cv[test_idx_local][feature_cols]
        y_te = df_cv[test_idx_local][TARGET]

        m = lgb.LGBMRegressor(**params)
        m.fit(X_tr, y_tr, sample_weight=w_tr)
        fold_maes.append(mean_absolute_error(y_te, m.predict(X_te)))

    return np.mean(fold_maes)

cv_study = optuna.create_study(direction='minimize')
cv_study.optimize(cv_objective, n_trials=300, show_progress_bar=True)

print(f"Best CV MAE : {cv_study.best_value:.3f}")
print(f"Best params : {cv_study.best_params}")

  0%|          | 0/300 [00:00<?, ?it/s]

Best CV MAE : 0.632
Best params : {'n_estimators': 544, 'learning_rate': 0.11418141890472619, 'max_depth': 5, 'num_leaves': 17, 'min_child_samples': 11, 'subsample': 0.6589433698971362, 'colsample_bytree': 0.9865453888942938, 'reg_alpha': 0.22478621228495935, 'reg_lambda': 0.5321437752310665}


In [22]:
# VERIFY FIT WITH CV PARAMS

fold_maes = []

for i, (year, test_end) in enumerate([(2023, 23), (2024, 25), (2025, 25)], 1):
    year_mask = df_2025['year'] == year

    df_cv = pd.get_dummies(df_2025[year_mask],
                           columns=['circuit_type', 'season_stage'],
                           prefix=['circuit', 'stage'])
    drop_cols_cv = [c for c in META_COLS + LEAKAGE_COLS + ['round']
                    if c in df_cv.columns]
    df_cv = df_cv.drop(columns=drop_cols_cv)

    train_idx = df_2025[year_mask]['round'].isin(range(1, 19)).values
    test_idx  = df_2025[year_mask]['round'].isin(range(19, test_end)).values

    X_tr = df_cv[train_idx][feature_cols]
    y_tr = df_cv[train_idx][TARGET]
    w_tr = df_2025[year_mask][train_idx]['sample_weight']
    X_te = df_cv[test_idx][feature_cols]
    y_te = df_cv[test_idx][TARGET]

    cv_model = lgb.LGBMRegressor(**cv_study.best_params, random_state=42, verbose=-1)
    cv_model.fit(X_tr, y_tr, sample_weight=w_tr)

    train_mae = mean_absolute_error(y_tr, cv_model.predict(X_tr))
    test_mae  = mean_absolute_error(y_te, cv_model.predict(X_te))
    gap       = test_mae - train_mae

    fold_maes.append(test_mae)
    print(f"Fold {i} ({year}) | Train MAE: {train_mae:.3f} | Test MAE: {test_mae:.3f} | Gap: {gap:.3f}")

print(f"\nAverage test MAE : {np.mean(fold_maes):.3f}")
print(f"Std deviation    : {np.std(fold_maes):.3f}")


Fold 1 (2023) | Train MAE: 0.077 | Test MAE: 0.645 | Gap: 0.568
Fold 2 (2024) | Train MAE: 0.047 | Test MAE: 0.771 | Gap: 0.725
Fold 3 (2025) | Train MAE: 0.053 | Test MAE: 0.481 | Gap: 0.428

Average test MAE : 0.632
Std deviation    : 0.119


In [23]:
# RETRAIN FINAL MODEL ON ALL DATA (2023+2024+2025)

df_full = pd.get_dummies(df_2025, columns=['circuit_type', 'season_stage'],
                         prefix=['circuit', 'stage'])

drop_cols_full = [c for c in META_COLS + LEAKAGE_COLS + ['round']
                  if c in df_full.columns]

w_full = df_2025['sample_weight']
df_full = df_full.drop(columns=drop_cols_full)

X_full = df_full[feature_cols]
y_full = df_full[TARGET]

final_model = lgb.LGBMRegressor(**cv_study.best_params, random_state=42, verbose=-1)
final_model.fit(X_full, y_full, sample_weight=w_full)

print(f"Final model trained on {len(X_full)} rows")
print(f"  2023: {(df_2025['year']==2023).sum()} rows (weight 0.5)")
print(f"  2024: {(df_2025['year']==2024).sum()} rows (weight 0.8)")
print(f"  2025: {(df_2025['year']==2025).sum()} rows (weight 1.0)")
print(f"Features used: {len(feature_cols)}")
print(f"Reported CV MAE: {cv_study.best_value:.3f}")


Final model trained on 1398 rows
  2023: 440 rows (weight 0.5)
  2024: 479 rows (weight 0.8)
  2025: 479 rows (weight 1.0)
Features used: 16
Reported CV MAE: 0.632


In [24]:
# SAVE MODEL

import pickle
import os

os.makedirs('../models', exist_ok=True)

# Save model
with open('../models/lgbm_regressor.pkl', 'wb') as f:
    pickle.dump(final_model, f)

# Save feature list — important for prediction later
with open('../models/feature_cols.pkl', 'wb') as f:
    pickle.dump(feature_cols, f)


# BUILD 2026 PREDICTION INPUT

# Our 2026 features map to 2025 training features like this:
# 2025 feature              ← 2026 replacement
# rolling_avg_finish_5      ← combined_pace_gap_s (car pace proxy)
# constructor_rolling_pts_5 ← team_reliability_score (team form proxy)
# avg_team_pit_seconds      ← stays same (pit crew unchanged)
# dnf_rate                  ← stays same (driver behaviour unchanged)
# quali_position            ← test2_gap_s (qualifying pace proxy)
# rolling_avg_quali_5       ← test2_gap_s (same signal)
# outperformance            ← stays same (driver talent unchanged)
# driver_experience         ← stays same (experience unchanged)
# teammate_quali_gap        ← stays same (relative pace unchanged)
# grid_position             ← test2_gap_s (starting position proxy)


In [25]:
# BUILD 2026 PREDICTION INPUT

df_26 = pd.read_csv('../data/features_2026.csv')

# Use only 2025 data for stable driver features
# (most recent season = most relevant baseline)
df_2025_only = df_2025[df_2025['year'] == 2025]

driver_stable = df_2025_only.groupby('driver').agg(
    outperformance        = ('outperformance', 'mean'),
    driver_experience     = ('driver_experience', 'mean'),
    avg_team_pit_seconds  = ('avg_team_pit_seconds', 'mean'),
    dnf_rate              = ('dnf_rate', 'mean'),
    teammate_quali_gap    = ('teammate_quali_gap', 'mean'),
    rolling_avg_finish_5  = ('rolling_avg_finish_5', 'mean'),
    rolling_avg_points_5  = ('rolling_avg_points_5', 'mean'),
    rolling_avg_quali_5   = ('rolling_avg_quali_5', 'mean'),
    quali_position        = ('quali_position', 'mean'),
    grid_position         = ('grid_position', 'mean'),
).reset_index()

# Load BOT and PER 2024 features
bot_per = pd.read_csv('../data/bot_per_2024_features.csv')

# Combine all 22 drivers
driver_all = pd.concat([driver_stable, bot_per], ignore_index=True)

# Merge with 2026 testing features
df_pred = df_26.merge(driver_all, on='driver', how='left')

print(f"Prediction input shape: {df_pred.shape}")
print(f"Drivers: {sorted(df_pred['driver'].tolist())}")
print(f"\nMissing values:")
print(df_pred.isnull().sum()[df_pred.isnull().sum() > 0])


Prediction input shape: (22, 29)
Drivers: ['ALB', 'ALO', 'ANT', 'BEA', 'BOR', 'BOT', 'COL', 'GAS', 'HAD', 'HAM', 'HUL', 'LAW', 'LEC', 'LIN', 'NOR', 'OCO', 'PER', 'PIA', 'RUS', 'SAI', 'STR', 'VER']

Missing values:
barcelona_best_s        3
barcelona_gap_s         3
outperformance          1
driver_experience       1
avg_team_pit_seconds    1
dnf_rate                1
teammate_quali_gap      1
rolling_avg_finish_5    1
rolling_avg_points_5    1
rolling_avg_quali_5     1
quali_position          1
grid_position           3
dtype: int64


In [26]:
# HANDLE MISSING VALUES

# Barcelona missing — fill with worst gap + penalty
worst_barcelona_gap = df_pred['barcelona_gap_s'].max()
df_pred['barcelona_gap_s']  = df_pred['barcelona_gap_s'].fillna(worst_barcelona_gap + 1.0)
df_pred['barcelona_best_s'] = df_pred['barcelona_best_s'].fillna(df_pred['barcelona_best_s'].max() + 1.0)


# LIN — true rookie, use conservative league averages
league_avg = driver_stable.mean(numeric_only=True)
rookie_cols = ['outperformance', 'driver_experience', 'avg_team_pit_seconds',
               'dnf_rate', 'teammate_quali_gap', 'rolling_avg_finish_5',
               'rolling_avg_points_5', 'rolling_avg_quali_5', 
               'quali_position', 'grid_position']

for col in rookie_cols:
    df_pred.loc[df_pred['driver'] == 'LIN', col] = league_avg[col]

# Override driver_experience for LIN manually
df_pred.loc[df_pred['driver'] == 'LIN', 'driver_experience'] = 0

# BOT and PER missing grid_position — use their quali_position
df_pred.loc[df_pred['driver'] == 'BOT', 'grid_position'] = \
    df_pred.loc[df_pred['driver'] == 'BOT', 'quali_position'].values[0]
df_pred.loc[df_pred['driver'] == 'PER', 'grid_position'] = \
    df_pred.loc[df_pred['driver'] == 'PER', 'quali_position'].values[0]

# Verify
print("Missing values after fix:")
missing = df_pred.isnull().sum()
print(missing[missing > 0] if missing[missing > 0].any() else "✅ No missing values!")

# Cap driver experience at 100
df_pred['driver_experience'] = df_pred['driver_experience'].clip(upper=100)
print(f"\nDriver experience after cap:")
print(df_pred[['driver', 'driver_experience']].to_string())

Missing values after fix:
✅ No missing values!

Driver experience after cap:
   driver  driver_experience
0     ALB              100.0
1     ALO              100.0
2     ANT                1.0
3     BEA               15.0
4     BOR                1.0
5     BOT              100.0
6     COL               10.0
7     GAS              100.0
8     HAD                1.0
9     HAM              100.0
10    HUL              100.0
11    LAW               20.0
12    LEC              100.0
13    LIN                0.0
14    NOR              100.0
15    OCO              100.0
16    PER              100.0
17    PIA               50.0
18    RUS              100.0
19    SAI              100.0
20    STR              100.0
21    VER              100.0


In [27]:
# BUILD CIRCUIT FEATURES FROM 2025 ONLY

circuit_features = df_2025[df_2025['year'] == 2025].groupby('race_name').agg(
    is_sprint_weekend      = ('is_sprint_weekend', 'max'),
    overtaking_difficulty  = ('overtaking_difficulty', 'first'),
    safety_car_probability = ('safety_car_probability', 'first'),
    circuit_type           = ('circuit_type', 'first'),
).reset_index()

print("2025 circuit features:")
print(circuit_features.to_string())


2025 circuit features:
                    race_name  is_sprint_weekend  overtaking_difficulty  safety_car_probability circuit_type
0        Abu Dhabi Grand Prix                  0                    2.0                     0.3    permanent
1       Australian Grand Prix                  0                    2.0                     0.7       street
2         Austrian Grand Prix                  0                    2.0                     0.5    permanent
3       Azerbaijan Grand Prix                  0                    2.0                     0.8       street
4          Bahrain Grand Prix                  0                    2.0                     0.3    permanent
5          Belgian Grand Prix                  0                    1.0                     0.6    permanent
6          British Grand Prix                  1                    2.0                     0.5    permanent
7         Canadian Grand Prix                  0                    3.0                     0.7    perman

In [28]:
# BUILD 2026 RACE CALENDAR


# 2026 F1 Calendar (24 rounds)
calendar_2026 = [
    {'round': 1,  'race_name': 'Australian Grand Prix'},
    {'round': 2,  'race_name': 'Chinese Grand Prix'},
    {'round': 3,  'race_name': 'Japanese Grand Prix'},
    {'round': 4,  'race_name': 'Bahrain Grand Prix'},
    {'round': 5,  'race_name': 'Saudi Arabian Grand Prix'},
    {'round': 6,  'race_name': 'Miami Grand Prix'},
    {'round': 7,  'race_name': 'Emilia Romagna Grand Prix'},
    {'round': 8,  'race_name': 'Monaco Grand Prix'},
    {'round': 9,  'race_name': 'Spanish Grand Prix'},
    {'round': 10, 'race_name': 'Canadian Grand Prix'},
    {'round': 11, 'race_name': 'Austrian Grand Prix'},
    {'round': 12, 'race_name': 'British Grand Prix'},
    {'round': 13, 'race_name': 'Belgian Grand Prix'},
    {'round': 14, 'race_name': 'Hungarian Grand Prix'},
    {'round': 15, 'race_name': 'Dutch Grand Prix'},
    {'round': 16, 'race_name': 'Italian Grand Prix'},
    {'round': 17, 'race_name': 'Azerbaijan Grand Prix'},
    {'round': 18, 'race_name': 'Singapore Grand Prix'},
    {'round': 19, 'race_name': 'United States Grand Prix'},
    {'round': 20, 'race_name': 'Mexico City Grand Prix'},
    {'round': 21, 'race_name': 'São Paulo Grand Prix'},
    {'round': 22, 'race_name': 'Las Vegas Grand Prix'},
    {'round': 23, 'race_name': 'Qatar Grand Prix'},
    {'round': 24, 'race_name': 'Abu Dhabi Grand Prix'},
]

df_calendar = pd.DataFrame(calendar_2026)

# Add season stage
df_calendar['stage_early'] = (df_calendar['round'] <= 8).astype(int)
df_calendar['stage_mid']   = ((df_calendar['round'] > 8) & (df_calendar['round'] <= 16)).astype(int)

# Merge circuit features
df_calendar = df_calendar.merge(circuit_features, on='race_name', how='left')

# One hot encode circuit_type
df_calendar = pd.get_dummies(df_calendar, columns=['circuit_type'], prefix='circuit')

# Check missing circuits
missing = df_calendar[df_calendar['overtaking_difficulty'].isna()]
print(f"Missing circuit data: {len(missing)} races")
if len(missing) > 0:
    print(missing[['round', 'race_name']])

print(f"\n2026 Calendar ({len(df_calendar)} races):")
print(df_calendar[['round', 'race_name', 'is_sprint_weekend', 
                    'overtaking_difficulty', 'safety_car_probability',
                    'stage_early', 'stage_mid']].to_string())

Missing circuit data: 0 races

2026 Calendar (24 races):
    round                  race_name  is_sprint_weekend  overtaking_difficulty  safety_car_probability  stage_early  stage_mid
0       1      Australian Grand Prix                  0                    2.0                     0.7            1          0
1       2         Chinese Grand Prix                  1                    2.0                     0.5            1          0
2       3        Japanese Grand Prix                  0                    3.0                     0.4            1          0
3       4         Bahrain Grand Prix                  0                    2.0                     0.3            1          0
4       5   Saudi Arabian Grand Prix                  0                    2.0                     0.8            1          0
5       6           Miami Grand Prix                  1                    3.0                     0.6            1          0
6       7  Emilia Romagna Grand Prix                  

In [31]:
# MONTE CARLO SIMULATION (10,000 runs)


N_SIMULATIONS = 10000
POINTS = {1:25, 2:18, 3:15, 4:12, 5:10, 6:8, 7:6, 8:4, 9:2, 10:1}

drivers = df_pred['driver'].tolist()
dnf_rates = df_pred.set_index('driver')['dnf_rate'].to_dict()

# STEP 1 — Get base predictions for all 24 races ONCE
base_preds = {}
for _, race in df_calendar.iterrows():
    race_df = df_pred.copy()
    race_df['is_sprint_weekend']            = race['is_sprint_weekend']
    race_df['overtaking_difficulty']        = race['overtaking_difficulty']
    race_df['safety_car_probability']       = race['safety_car_probability']
    race_df['stage_early']                  = race['stage_early']
    race_df['stage_mid']                    = race['stage_mid']
    race_df['grid_position']               = 1 + (race_df['combined_pace_gap_s'] / 0.3)
    race_df['quali_position']              = race_df['grid_position']
    race_df['rolling_avg_quali_5']         = race_df['grid_position']
    race_df['rolling_avg_finish_5']        = race_df['grid_position']
    race_df['constructor_rolling_points_5'] = race_df['team_reliability_score'] * 100
    base_preds[race['round']] = final_model.predict(race_df[feature_cols])

print("✅ Base predictions done (24 races)")

# STEP 2 — Monte Carlo in pure numpy (fast)
sim_points = np.zeros((N_SIMULATIONS, len(drivers)))
sim_wins   = np.zeros(len(drivers))

dnf_array = np.array([dnf_rates[d] for d in drivers])

for sim in range(N_SIMULATIONS):
    season_pts = np.zeros(len(drivers))

    for rnd, base in base_preds.items():
        # Add noise
        noise_scale = 0.803 + (race['safety_car_probability'] * 2.0)
        noise = np.random.normal(0, noise_scale, size=len(base))
        preds = base + noise

        # Simulate DNFs
        dnf_mask = np.random.random(len(drivers)) < dnf_array
        preds[dnf_mask] += 15

        # Rank top 20
        ranked_idx = np.argsort(preds)[:20]
        for pos, idx in enumerate(ranked_idx, 1):
            season_pts[idx] += POINTS.get(pos, 0)

        sim_wins[ranked_idx[0]] += 1

    sim_points[sim] = season_pts

print(f"✅ {N_SIMULATIONS:,} simulations complete!")

# SUMMARISE
results = []
for i, driver in enumerate(drivers):
    results.append({
        'driver'    : driver,
        'avg_points': round(sim_points[:, i].mean(), 1),
        'min_points': round(sim_points[:, i].min(), 1),
        'max_points': round(sim_points[:, i].max(), 1),
        'win_pct': round(sim_wins[i] / (N_SIMULATIONS * 24) * 100, 1)
    })

df_sim = pd.DataFrame(results).sort_values('avg_points', ascending=False).reset_index(drop=True)
df_sim['position'] = range(1, len(df_sim) + 1)

print("\n*** 2026 MONTE CARLO CHAMPIONSHIP PREDICTION ***")
print(df_sim[['position', 'driver', 'avg_points', 'win_pct', 
              'min_points', 'max_points']].to_string(index=False))

✅ Base predictions done (24 races)
✅ 10,000 simulations complete!

*** 2026 MONTE CARLO CHAMPIONSHIP PREDICTION ***
 position driver  avg_points  win_pct  min_points  max_points
        1    LEC       410.2     40.8       245.0       538.0
        2    RUS       348.7     16.8       236.0       474.0
        3    PIA       331.0     15.7       214.0       454.0
        4    VER       298.6      8.9       194.0       397.0
        5    NOR       272.4      8.2       143.0       388.0
        6    HAM       251.1      7.0       136.0       377.0
        7    ANT       181.2      2.4        77.0       293.0
        8    BEA       105.9      0.1        36.0       182.0
        9    ALB        50.2      0.0        12.0       107.0
       10    OCO        45.6      0.0         7.0       107.0
       11    HUL        34.8      0.0         4.0        88.0
       12    HAD        22.6      0.0         0.0        61.0
       13    LIN        22.1      0.0         0.0        61.0
       14    SAI

In [32]:
# CELL 20 — MONTE CARLO CONSTRUCTORS CHAMPIONSHIP

# Team map from 2026 data
team_map = df_pred.set_index('driver')['team_2026'].to_dict()

# Add team to simulation results
# We need to rebuild from sim_points array

constructor_points = {}
for i, driver in enumerate(drivers):
    team = team_map[driver]
    if team not in constructor_points:
        constructor_points[team] = np.zeros(N_SIMULATIONS)
    constructor_points[team] += sim_points[:, i]

# Summarise constructors
constructor_results = []
for team, points_array in constructor_points.items():
    constructor_results.append({
        'team'        : team,
        'avg_points'  : round(points_array.mean(), 1),
        'min_points'  : round(points_array.min(), 1),
        'max_points'  : round(points_array.max(), 1),
        'champion_pct': round(np.mean(points_array == max(
                            v.mean() for v in constructor_points.values()
                        )) * 100, 1)
    })

df_constructors = pd.DataFrame(constructor_results)\
    .sort_values('avg_points', ascending=False).reset_index(drop=True)
df_constructors['position'] = range(1, len(df_constructors) + 1)

print("*** 2026 PREDICTED CONSTRUCTORS CHAMPIONSHIP ***")
print(df_constructors[['position', 'team', 'avg_points', 
                         'min_points', 'max_points']].to_string(index=False))

*** 2026 PREDICTED CONSTRUCTORS CHAMPIONSHIP ***
 position           team  avg_points  min_points  max_points
        1        Ferrari       661.4       436.0       831.0
        2        McLaren       603.5       452.0       759.0
        3       Mercedes       529.9       382.0       678.0
        4       Red Bull       321.2       224.0       430.0
        5   Haas F1 Team       151.5        71.0       244.0
        6       Williams        63.6        19.0       137.0
        7           Audi        38.6         6.0       101.0
        8     RB F1 Team        34.4         7.0        83.0
        9 Alpine F1 Team        15.0         0.0        51.0
       10   Aston Martin         2.7         0.0        25.0
       11       Cadillac         2.2         0.0        18.0


# Model 2 - Podium Classifier